# TicTacToe

In [2]:
from collections import namedtuple
import random
import math

In [3]:
GameState = namedtuple('GameState','to_move, utility, board, moves')

In [4]:
class TicTacToe:
    def __init__(self):
        moves = [(0, 0), (0, 1), (0, 2), 
                 (1, 0), (1, 1), (1, 2), 
                 (2, 0), (2, 1), (2, 2)]
        self.initial = GameState(to_move="X", utility=0, board={}, moves=moves)

    def actions(self, state):
        # Returns legal moves (i.e. empty spaces)
        return state.moves

    def player(self, state):
        return state.to_move

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move

        # Player to make a move
        player = self.player(state)

        # Board next turn
        board_next_turn = state.board.copy()
        board_next_turn[move] = player

        # Available moves next turn
        moves_next_turn = list(state.moves)
        moves_next_turn.remove(move)

        # Player to make a move next turn
        if player == "X":
            player_next_turn = "O"
        else:
            player_next_turn = "X"

        # Whether the player wins the game after move
        win = self.three_in_row(board_next_turn, move, player)

        # Utility of the next state of the game
        if not win:
            utility_next_turn = 0
        elif player == "X":
            utility_next_turn = 1
        else:
            utility_next_turn = -1

        return GameState(
            to_move=player_next_turn,
            utility=utility_next_turn,
            board=board_next_turn,
            moves=moves_next_turn,
        )

    def terminal(self, state):
        return state.utility != 0 or len(state.moves) == 0

    def three_in_row(self, board, move, player):
        # Returns true if player has three pieces in a line through square
        for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)):

            n = 0
            x, y = move
            while board.get((x, y)) == player:
                n = n + 1
                x = x + dx
                y = y + dy

            x, y = move
            while board.get((x, y)) == player:
                n = n + 1
                x = x - dx
                y = y - dy

            n = n - 1

            if n >= 3:
                return True

        return False

    def utility(self, state, player):
        if player == "X":
            return state.utility
        else:
            return -state.utility

    def display(self, state):
        board = state.board
        for x in range(3):
            for y in range(3):
                print(board.get((x, y), "."), end=" ")
            print()

    def play_game(self, player1, player2):
        state = self.initial
        players = [player1, player2]
        while True:
            for player in players:
                print("-----------------------")
                cur_player = self.player(state)
                print(cur_player, "plays")
                move = player(self, state)
                state = self.result(state, move)
                self.display(state)
                if self.terminal(state):
                    print()
                    print("###########################")
                    if self.utility(state, player) == 0:
                        print("Draw")
                    else:
                        print(cur_player, "wins!!!")
                    print("###########################")
                    return True

### Player - Random Player and Minimax Player
Below are the functions that represents player agents that plays randomly and those that play with minimax algortihm. 

In [5]:
infinity = math.inf

In [6]:
def random_player(game, state):
    if game.actions(state):
        return random.choice(game.actions(state))
    else:
        return None

In [7]:
def minmax_player(game, state):
    player = game.player(state)
    
    def max_value(state):
        if game.terminal(state):
            return game.utility(state,player)
        v = -infinity 
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v
            
    def min_value(state):
        if game.terminal(state):
            return game.utility(state,player)
        v = infinity 
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v
    
    best_score = -infinity 
    best_action = None

    for a in game.actions(state):
        v = min_value(game.result(state,a))
        if v > best_score:
            best_score = v
            best_action = a
    return best_action
  

### Game
To play a game, we will create a TicTacToe object from the class we defined above. Below we create one minmax and one random player and we pass these players to `play_game(player1, player2)` method to make them play to each other.

In [12]:
ttt = TicTacToe()
p1 = minmax_player
p2 = random_player
ttt.play_game(p1,p2)

-----------------------
X plays
X . . 
. . . 
. . . 
-----------------------
O plays
X . . 
. . . 
. . O 
-----------------------
X plays
X . X 
. . . 
. . O 
-----------------------
O plays
X . X 
. . O 
. . O 
-----------------------
X plays
X X X 
. . O 
. . O 

###########################
X wins!!!
###########################


True

### Alpha Beta Pruning

Let us implement alpha beta pruning algortihm to the function below. 

A minmax player playing against a random player.

In [13]:
def alphabeta_player(game, state):
    player = game.player(state)
    
    def max_value(state, alpha, beta):
        if game.terminal(state):
            return game.utility(state,player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state,a), alpha, beta))
            if v >= beta: 
                return v
            alpha = max(alpha, v)
        return v
            
    def min_value(state, alpha, beta):
        if game.terminal(state):
            return game.utility(state,player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a),alpha,beta))
            if v <= alpha:
                return v            
            beta = min(beta, v)
        return v
    
    best_score = -infinity
    best_action = None
    
    for a in game.actions(state):
        v = min_value(game.result(state,a), -infinity, infinity)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

After implementing alpha beta algorithm, we can make it play against a random player by running the code below.

In [14]:
ttt = TicTacToe()
p1 = alphabeta_player
p2 = random_player
ttt.play_game(p1,p2)

-----------------------
X plays
X . . 
. . . 
. . . 
-----------------------
O plays
X . . 
. O . 
. . . 
-----------------------
X plays
X X . 
. O . 
. . . 
-----------------------
O plays
X X . 
. O . 
. . O 
-----------------------
X plays
X X X 
. O . 
. . O 

###########################
X wins!!!
###########################


True